In [11]:
import numpy as np

In [98]:
# given a (set of) polymer structure, create a set of M^(2) connectivity matrices

poly = [1, 1, 0, 0]
n_p = 1
M = len(poly)
alph1 =1
alph2 =0

mat = np.zeros((M, M)) #MxM 

for j1,M1 in enumerate(poly):
    for j2,M2 in enumerate(poly):
#         print("M1 = ", M1)
#         print(M2)
#         print((M1 == alph1) * (M2 == alph2))
        mat[j1, j2] += (1/n_p) * (M1 == alph1) * (M2 == alph2)# TODO how to implement indicator vairalbe
        
#TODO consider multiple polymer types

In [99]:
mat

array([[0., 0., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [221]:
#vectorized single poly


poly = np.array([1, 1, 0, 0])
n_p = 1
M = len(poly)
alph1 =np.ones(M)
alph2 =np.ones(M)
sub1 = 1*(poly == alph1)
sub2 = 1*(poly == alph2)
mat_Vect = np.outer(sub1, sub2)

In [222]:
mat_Vect

array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [189]:
#vectorized multi poly. inly considering equal amounts of each polymer. TODO- consider unequal amounts. Manipulate simga?

poly_mat = np.array([[1, 0], [0, 1], [1, 1]]) #matrix where num rows = num polymers
# poly_mat = np.array([[0, 0],[1, 1]]) #matrix where num rows = num polymers

# poly_mat = np.array([1, 0])

#TODO: handle special case where polymat only has one polymer
#then implement previous algorithm; have alph1 and alph2 be 1D 
n_p = poly_mat.shape[0]
M = poly_mat.shape[1]
# alph1 =np.zeros((M, M))
# alph2 =np.zeros((M, M))
alph1 =np.ones((n_p, M))
alph2 =np.ones((n_p, M))


sigma1 = 1*((poly_mat == alph1))#.sum(axis = 0) #sigma. could multiply each
sigma2 = 1*((poly_mat == alph2))#.sum(axis = 0)

# mat_Vect_two = np.matmul(sub1, sub2) * (1/n_p)

#need to do each row outer product with corresponding row, get n_p MxM matrices, then sum the results
prods = np.einsum('bi,bo->bio', sigma1, sigma2) # performing row wise cross product (each poly contribution)
mat_Vect_two = np.sum(prods, axis = 0) / n_p # averaging each contribution

In [229]:
# Have M. Now need to multiple upper triangle, lower triangle, and diagonal by sf components (construct C matrix, mat mul the)
#  (the debye func at diag, and the apprprt exp(-xm j(1or2) - j (2or1)) * 1/xm2 blah blah)
N_m = 1
b = 1
k = 1#np.array([1, 2, 3])

x_m = (1/6) * N_m * b**2 * k**2

C = np.zeros((M, M))
one = np.ones((M,M))
grid = np.indices((M, M))


j1 = j2
debye = (2/(x_m**2)) * (x_m + np.exp(-x_m) - 1) 
np.fill_diagonal(C, np.diag(one * debye))

integral = (1/(x_m**2)) * (np.exp(x_m) + np.exp(-x_m) - 2) #for off-diagonal terms

#j1 > j2
j1minusj2_mat = grid[0] - grid[1]
C += np.tril(one * np.exp(-x_m*(j1minusj2_mat)) * integral, k = -1)

#j2 > j1
j2minusj1_mat = -1 * j1minusj2_mat
C += np.triu(one * np.exp(-x_m*(j2minusj1_mat)) * integral, k = 1)
 
#isnt j2minusj1 and j1minusj2 the same?? since only done in special case where the one is bigger than the other?

In [230]:
S2_AA = np.sum(C * mat_Vect)

In [231]:
S2_AA

3.5902543613128177

In [ ]:
# compare to Vertex/wlcstat codebase at discrete ks
# vectorize wrt k !

In [ ]:
#Now derive conversion rules from AA to AB and BB (G_AA, GAB, G_BB in notes. Re derive!!!!!). 
#Compare to vertex/wlcstat!!
